# Number Crunching with Dask Array

We'll be helping the zoning and land-use departments choose locations for new recreation facilities in town. For that, we'll need to analyze array data reflecting the local topography.

This will involve
* understanding array data
* loading relevant datasets with Dask array
* analyzing the data to locate key sites
* writing the processes data to a suitable format

## What is array data?

If you're a scientist or data analyst who works with array data extensively, this part won't be too surprising. But for folks who typically work with tabular data, a brief intro to array data will get us ready for Dask Array

__Array data__ typically refers to a rectangular, multidimensional collection of values, all of the same type.

Multidimensional is usually, though not always, small (say, 2-5) and the data type is usually, though not always, numeric

In Python, we typically use the NumPy library to work with this sort of data, and the array data structure is called NDArray

In [ ]:
import numpy as np

np.eye(3)

In [ ]:
np.random.normal(0, 1, (2,2))

### Underneath the hood, a lot of tabular data is array data

For example, we might start with a Pandas data table and featurize it for machine learning, producing a table of numbers.

In [ ]:
import pandas as pd

df = pd.DataFrame({'league':['B', 'B', 'A', 'C'], 'games':[34,22,66,12]})

prepared = pd.get_dummies(df)

prepared

In [ ]:
prepared.values

In [ ]:
prepared.values.dtype

In many cases, where our data consist completely of numeric measurements, we go straight to the array representation

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

iris.data[:5]

In [ ]:
iris.feature_names

These features have specific, meaningful names, but often we have data where that isn't the case.

For example, the pixels in this image don't have meaningful names; they make more sense as an array

<img src='images/trees.png'>

We can load and manipulate them as array data

In [ ]:
import imageio
import matplotlib.pyplot as plt

imagedata = imageio.imread('images/trees.png')

print(type(imagedata), imagedata.dtype, imagedata.shape)

In [ ]:
plt.imshow(imagedata)

There is an alpha channel ... does it have any data?

In [ ]:
plt.imshow(imagedata[:,:,3])

Let's drop the alpha channel and convert to grayscale

In [ ]:
no_alpha = imagedata[:,:,:3]

no_alpha.shape

In [ ]:
grayscale = no_alpha.mean(axis=2)

grayscale.shape

In [ ]:
plt.gray()

plt.imshow(grayscale)

In this example, we started with 3 axes, sliced to discard the alpha channel, then averaged over the color axis to get approximate intensity, and ended with a grayscale matrix.

It's easy to imagine large datasets where we might want to do similar operations
* 10 minutes of 4K video at 60Hz might look like an array of shape (2160, 3840, 3, 36000) and consume almost 1TB uncompressed
* Even at low resolution, global satellite imagery data, atmospheric data, ocean data, and similar datasets will likely be big

## Ok, now let's bring Dask into the discussion

In [ ]:
import coiled
from dask.distributed import Client

cluster = coiled.Cluster(name="training-cluster")
client = Client(cluster)
client

__Array data at rest__

In some domains, we may come across array data stored as, say, CSV files or pickled Torch Tensors, but often we'll work with dedicated formats specifically designed around array datasets.

One popular format is HDF5
* https://www.neonscience.org/about-hdf5
* https://en.wikipedia.org/wiki/Hierarchical_Data_Format

Dask array can consume these in a lazy, parallel manner ... it would look like this:

```python
import h5py
import dask.array as da

file = h5py.File('datafile.h5', 'r')
dataset = file['dataset_within_file']
arr = da.from_array(dataset)
```

The challenge comes when working with cloud data storage. The Python integrations to cloud storage a bit quirky or problematic when processing HDF5. For some datasets, it might work, and would look like:

```python
import s3fs

s3 = s3fs.S3FileSystem() # credentials should be in env before this point
file = h5py.File(s3.open('s3://bucket/path/data.h5', 'rb'))

dataset = file['dataset_within_file']
arr = da.from_array(dataset)
```

To avoid those quirks/issues, though, we'll work with a more modern and efficient format, in terms of both compression and usefulness for parallel computation, called Zarr
* https://zarr.readthedocs.io/en/stable/

In [ ]:
import dask.array as da

arr = da.from_zarr('s3://coiled-training/data/land', storage_options={"anon": True})

arr

That seemed to work ... and we get a nice HTML view of our 3-axis dataset.

But there are a couple of problems. 

First, this seems like at least an axis too many: after all, how many elevations can a single coordinate pair on earth have? We'll come back to that part -- how to decode the data. It's not uncommon to need to decode datasets in a situation like this.

Next, and more severe: if we try to operate on this dataset right now, 
* the whole thing will end up getting loaded in one worker (if it fits)
* we won't get any parallelism

*Unlike Dask dataframe, Dask array does not automatically create partitions*

We need to give it some hints. Dask array partitions are called *chunks* or *blocks* and they're a little different from Dask dataframe partitions.

Here's what a Dask array looks like, conceptually

<img src='images/dask-array.svg' width=600>

A Dask array is composed of multiple "regular" NumPy NDArray chunks... but they can be divided into pieces along all of the axes.

From the documented best practices, 
* You want to choose a chunk size that is large in order to reduce the number of chunks that Dask has to think about (which affects overhead)
* but also small enough so that many of them can fit in memory at once.
* Dask will often have as many chunks in memory as twice the number of active threads.

In a real application, this typically means chunks of 100MB or even much more, but for our exercise we'll have smaller data and much smaller chunks.

Let's try dividing on the first two axes (roughly speaking, latitude and longitude) but not the third ("depth") axis.

In [ ]:
arr = da.from_zarr('s3://coiled-training/data/land', chunks=(200, 200, 4), storage_options={"anon": True})

arr

Here we can see stats for the overall dataset under the Array column, while stats for each block appear under the Chunk column.

The visualization is helpful as well, although in high numbers of dimensions (axes) that might prove less useful.

The blocks are indexed along all axes, and we can access them (if needed) in a similar way to the partitions of Dask dataframe.

In [ ]:
arr.blocks[0, 0]

In [ ]:
type(arr.blocks[0,0])

In [ ]:
arr.blocks[0,0].compute()

In [ ]:
type(arr.blocks[0,0].compute())

__Note:__ Dask arrays also have a `chunks` property, which is a different: `chunks` provides tuples of sizes for the individual blocks

In [ ]:
arr.chunks

That seems kind of silly at first, because we told Dask how to chunk the array in the first place, so shouldn't these sizes match?

Not necessarily. In our example, the chunks divide the dataset evenly. But that doesn't have to be the case.

In [ ]:
uneven = da.random.uniform(0, 10, size=(17, 23, 29), chunks=(5,7,11))
uneven

In [ ]:
uneven.chunks

In this case the chunk dimensions don't evenly divide the dataset. While not ideal, this may make sense in a lot of situations. Perhaps the main task we're performing for our work naturally occurs on blocks of size 17x23x29. In that case, the uneven chunking may allow more efficiency overall.

The critical point is that 
* the `blocks` property exposes delayed references to actual blocks of data; 
* the `chunks` property exposes the actual chunk dimensions along each axis, while 
* the `chunksize` property stores the requested or target size of a chunk,
    * even if not every chunk matches this size.

In [ ]:
uneven.chunksize

## API

The Dask array API exposes most, though not all, of the NumPy API. So we can use our NumPy knowledge to apply operations to our data.

In the land elevation dataset, the data are actually image tiles, where the fourth axis encodes color and alpha information.

The alpha channel should be uninformative... let's see:

In [ ]:
arr[:, :, 3]

In [ ]:
arr[:, :, 3].min()

In [ ]:
arr[:, :, 3].min().compute()

In [ ]:
arr[:, :, 3].max().compute()

Dask collections -- like Array and Dataframe -- "hide" their task graph information in a member called `dask`

Until recently, inspecting the Dask graph was tedious and difficult, since it included a very large number of tasks. Recall that a Dask task is just a Python function, so it's often just one step of a calculation on just one chunk of a data structure -- even one calculation, if it's to be performed over 1,000 blocks, would mean 1,000 tasks will be in the full graph.

Recently, a series of under-the-hood improvements to Dask performance have also made it easier to inspect Dask computations by exposing the *high-level graph*. Each item in the high-level graph is typically a single calculation -- but applied to all of the blocks, not just one. That means high-level graphs are much smaller, making them more friendly for the Dask `Client` to work with, and for humans.

We can inspect the high-level graph corresponding to that last computation like this. (If you like seeing what Dask plans to do with your data, you can back and take a look at `.dask` on some of our dataframe examples as well.)

In [ ]:
arr[:, :, 3].max().dask

The high-level graph visualization gives a static view of the planned work. But we can also see a dynamic, "live" view that corresponds to this coarse-grained execution by opening the __Groups__ (short for "Task Groups Graph") dashboard. Open that up and run the calculation again.

In [ ]:
arr[:, :, 3].max().compute()

That was a bit quick -- but for your longer, larger data processing work, you'll have time to inspect the graph and observe the progress animation to see how it is going.

For now, to "freeze" the graph, we can cache our Dask array following the same API pattern we used for caching dataframes.

In [ ]:
cached_example = arr[:, :, 3].max().persist() # graph should remain in the Groups dashboard

As for our topography data, we've learned two things: indeed, the alpha channel has no signal for us. We can get rid of it.

More importantly, Dask array follows the same convention we've seen with dataframe and our own delayed objects: they are lazy by default, and we can use `.compute` when we want a result that is 
* a regular Python object
* completely loaded in our local Python process

As with dataframe, when we are producing a large-scale result, we probably want to either keep a handle (delayed) that we can perform more operations on, or, if we're finished, write it out in parallel using the cluster.

Let's keep just the RGB image channels and write it out -- note that Zarr format preserves our partitioning scheme.

In [ ]:
import os

bucket = os.environ['WRITE_BUCKET']

In [ ]:
arr[:, :, :3].to_zarr('s3://' + bucket + '/elevations', overwrite=True)

When we load this data, the structure is preserved

In [ ]:
elevations = da.from_zarr('s3://' + bucket + '/elevations')
elevations

The Dask array documentation is at https://docs.dask.org/en/latest/array.html

Let's run a couple of more expensive computations -- just toy calculations for now -- and introduce two new dashboards.

__Aggregate Time Per Action__ dashboard summarizes the time spent on categories of operations, like *compute*, *data transfer*, and *serialization* rather than at the level of individual tasks. For a high level performance view, this can be a better place to start than looking at time spend on specific tasks.

With that new dashboard open as well as the Task Stream, let's do some matrix multiplication and then add all of the resulting values.

Keep an eye on both dashboards.

In [ ]:
(elevations[:, :, 0] @ elevations[:, :, 1].transpose()).sum().compute()

This demo illustrates how the aggregate time report summarizes a lot of lower-level complexity in the task stream (as well as doing the arithmetic on total time for you).

Open the __Compute Time Per Key__ dashboard, which presents the expensive tasks summarized in a bar or pie chart. You should easily see that the `matmul-sum` task used the biggest chunk of time. That information -- and the color coding of the task bar itself -- should match what you're seeing in the Task Stream.

### Time to do some urban planning

Let's try and decode the elevation grid.

First we'll do the quick-and-dirty recipe (we'll save the precise calculation for our lab exercises).

The pixel instensity at a point corresponds to the relative elevation, scaled between 0 and (approximately) 1000 feet above sea level.

You can almost see the valley

In [ ]:
plt.imshow(elevations.blocks[0,0].compute()[100:, 100:])

Like with NumPy, we can average across the depth (RGB) axis to get a 2-d map of elevations

In [ ]:
elevation_map = elevations.mean(axis=2)
elevation_map

Now let's scale to get feet

In [ ]:
scaled_elevation_map = 1000.0 * elevation_map / 255
scaled_elevation_map

We need to find the area with the least rugged terrain for some campuses, office parks, and recreational spaces.

Let's start by looking for minimal variance within a block.

Dask array has a convenient API for running an operation for every block. Since we may want to transform the existing block to some new value, it's called `map_blocks` and takes a transforming function. In our case, we can use NumPy's `std` 

... but there's one small gotcha with `map_blocks`: it expects the output from the mapping (transforming) function to be the same shape (axes/size) as the input.

So we'll dress up our scalar output as a NDArray of shape (1,1)

In [ ]:
def elevation_std(array_block):
    return np.array([[np.std(array_block)]])

variance_da = scaled_elevation_map.map_blocks(elevation_std, chunks=(1,1))

variance_da

In [ ]:
variance = variance_da.compute()

In this case, we can visually inspect to see where the terrain might be smoothest ... but if we had a bigger dataset we might need to find that block programmatically or by visualizing

In [ ]:
plt.imshow(variance)

We can quickly see that block 2,5 is worth a look

In [ ]:
target_block = scaled_elevation_map.blocks[2,5].compute()

print(target_block.min(), target_block.mean(), target_block.max())

This areas looks promising so let's start from the top, improve our work so far, and then go a bit deeper.

## Lab: Investigating topography

#### Activity 1: Proper decoding of image data

Since human eyes have different sensitivities to red, green, and blue, a grayscale image is usually not encoded in RGB format using the same intensities (pixel values) for all 3 channels. In fact, even averaging across the channels doesn't always get you the right grayscale level.

This terrain data was encoded using the "luminosity" method, which means our real values are determined by this formula: 0.21 R + 0.72 G + 0.07 B

Use that formula to convert the 3-channel data to an elevation map. Then scale the elevations: 
* reference values for min and max elevation for this map area are 30 feet (min) to 1100 feet (max)
* because of how the maps are generated, those reference elevations may not actually exist in this area

Let's transform and then find the min/max in our land area

#### Activity 2: Finding least-rugged terrain block

Let's repeat our elevation variance investigation. See if you can also determine the coordinates of the "least-variance" block programmatically using Dask (not NumPy).

Hint: if we want Dask to know the shape of our array after calling `map_blocks`, we have to give that API call some info by using the `chunks=` parameter (since otherwise it has no idea of the shape our `elevation_std` will return)

#### Activity 3: Zoom in for smoother land

*Bonus Project*

Now that we know roughly where to search, let's zoom in on that specific block in the elevation map. It's 200x200 units, and those units are pretty big. Let's further divide that block into 10x10 areas and look again. Hint: find the original block and `rechunk`

For more information on how the elevation changes, try calculating `min` or `max` for the chunks and then generating a report of changes with `diff`

#### Lab Postscript

Another way to investigate these sorts of questions would be to perform some analysis for every group of elements -- e.g., a point and its neighborhood -- even when those points are on the edge of a block. 

Dask array has an API, `map_overlap`, that allows us to operate over a block but also have access to a small group of neighboring values that adjoin the block edges. Using this facility, we can do things like perform convolution/cross-correlations, create sliding (windowed) calculations like moving averages, approximate gradients ... which might be really helpful in finding smooth land!

More details at https://docs.dask.org/en/latest/array-overlap.html

## Best Practices

As with Dask dataframe, you can cache results using `persist`

Some additional best practices are at https://docs.dask.org/en/latest/array-best-practices.html

Both Zarr and HDF5 are solid formats for array data; avoid formats that don't align with the scalable reading and writing of arrays, like text-based or columnar formats.

## Swappable Implementations

Much of Dask array can be applied to alternative implementations of the NumPy interface

Among such implementations, the GPU-backed CuPy arrays (originally part of the Chainer project) are of particular interest: https://docs.dask.org/en/latest/gpu.html

Additional, even newer, implementations may be usable in the future, including
* JAX - jax.numpy https://jax.readthedocs.io/en/latest/jax.numpy.html
* TensorFlow - tf.experimental.numpy https://www.tensorflow.org/api_docs/python/tf/experimental/numpy